In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn

STATE_FILE = './chain-state.ndjson'

MINER_STATE_COL_RENAMES = {
    'Info.MinerAddr': 'Miner',
    'Info.MinerPower.MinerPower.RawBytePower': 'Info.MinerPowerRaw',
    'Info.MinerPower.MinerPower.QualityAdjPower': 'Info.MinerPowerQualityAdj',
    'Info.MinerPower.TotalPower.RawBytePower': 'Info.TotalPowerRaw',
    'Info.MinerPower.TotalPower.QualityAdjPower': 'Info.TotalPowerQualityAdj',
}

MINER_NUMERIC_COLS = [
    'Info.MinerPowerRaw',
    'Info.MinerPowerQualityAdj',
    'Info.TotalPowerRaw',
    'Info.TotalPowerQualityAdj',
    'Info.Balance',
    'Info.CommittedBytes',
    'Info.ProvingBytes',
    'Info.FaultyBytes',
    'Info.FaultyPercentage',
    'Info.PreCommitDeposits',
    'Info.LockedFunds',
    'Info.AvailableFunds',
    'Info.WorkerBalance',
    'Info.MarketEscrow',
    'Info.MarketLocked',
]

DERIVED_COLS = [
    'CommittedSectors',
    'ProvingSectors',
]

ATTO_FIL_COLS = [
    'Info.Balance',
    'Info.PreCommitDeposits',
    'Info.LockedFunds',
    'Info.AvailableFunds',
    'Info.WorkerBalance',
    'Info.MarketEscrow',
    'Info.MarketLocked',
]

def atto_to_fil(x):
    return float(x) * pow(10, -18)

def chain_state_to_pandas(statefile):
    chain = None
    
    with open(statefile, 'rt') as f:
        for line in f.readlines():
            j = json.loads(line)
            chain_height = j['Height']
            
            miners = j['MinerStates']
            for m in miners.values():
                df = pd.json_normalize(m)
                df['Height'] = chain_height
                df.rename(columns=MINER_STATE_COL_RENAMES, inplace=True)
                if chain is None:
                    chain = df
                else:
                    chain = chain.append(df, ignore_index=True)
    chain.fillna(0, inplace=True)
    chain.set_index('Height', inplace=True)
    
    for c in ATTO_FIL_COLS:
        chain[c] = chain[c].apply(atto_to_fil)
    
    for c in MINER_NUMERIC_COLS:
        chain[c] = chain[c].apply(pd.to_numeric)
            
    # the Sectors.* fields are lists of sector ids, but we want to plot counts, so
    # we pull the length of each list into a new column
    chain['CommittedSectors'] = chain['Sectors.Committed'].apply(lambda x: len(x))
    chain['ProvingSectors'] = chain['Sectors.Proving'].apply(lambda x: len(x))
    return chain
        
cs = chain_state_to_pandas(STATE_FILE)

In [ ]:
# choose which col to plot using a widget

cols_to_plot = MINER_NUMERIC_COLS + DERIVED_COLS

col_selector = pn.widgets.Select(name='Field', options=cols_to_plot)
cols = ['Miner'] + cols_to_plot
plot = cs[cols].hvplot(by='Miner', y=col_selector)
pn.Column(pn.WidgetBox(col_selector), plot)

In [ ]:
# plot all line charts in a vertical stack

plots = []
for c in cols_to_plot:
    title = c.split('.')[-1]
    p = cs[['Miner', c]].hvplot(by='Miner', y=c, title=title)
    plots.append(p)
pn.Column(*plots)

In [ ]:
# miner power area chart

mp = cs[['Miner', 'Info.MinerPowerRaw']].rename(columns={'Info.MinerPowerRaw': 'Power'})
mp = mp.pivot_table(values=['Power'], index=cs.index, columns='Miner', aggfunc='sum')
mp = mp.div(mp.sum(1), axis=0)
mp.columns = mp.columns.get_level_values(1)
mp.hvplot.area(title='Miner Power Distribution')